# Imports

In [4]:
from spark_tools import *
#from helio_utils.cu_google_sheet import cu_read_google_sheet

import pandas as pd
pd.set_option('display.max_rows', 200) # first 100 last 100

import apollo
from apollo_artifacts import datasets
from core.pandas_display_util import set_no_pandas_truncation_options
apollo.OverrideConfiguration().apply()

#from helio_utils.spark import spark_to_pandas
%config Completer.use_jedi = False


# Accesing Data
We use Apollo for accessing most of our s3 data. Apollo provides an access layer for production Helio data assets. You can see the list of datasets here:

https://github.com/CircleUp/apollo-artifacts/blob/master/catalog.md

or by using datasets. *tab*

In [5]:
# Prod datasets
apollo_data = apollo.show_all_datasets()
apollo_data.loc[(apollo_data.environment == "production")]

,name,LogicalDataset.created_by,LogicalDataset.created_on,published_to_production_on,segment_id,schema_name,PhysicalDataset.created_by,PhysicalDataset.created_on,environment,location,user
5,twitter__derived_growth__2_0,production,2019-08-23 21:09:21.140715,2022-03-08 14:22:34.147397,"{'_v': 1, '_suid': 115392967149, 'components': [{'_v': 1, 'name': 'execution_period', '_suid': 6968176}], 'path_temp...",twitter__derived_growth__002,production,2019-08-26 19:42:17.723668,production,hive://apollo_calculated.twitter__derived_growth__2_0,production
7,facebook_post__3_0,production,2019-08-21 21:00:16.027191,2022-03-08 14:22:00.157057,"{'_v': 1, '_suid': 266491286929, 'components': [{'_v': 1, 'name': 'source_name', '_suid': 1978769}, {'_v': 1, 'name'...",facebook_post__003,production,2019-08-21 21:00:16.027191,production,hive://apollo_normalized.facebook_post__3_0,production
8,facebook_page_brand_map__3_0,production,2019-08-21 21:00:16.652084,2022-03-08 14:21:57.325319,"{'_v': 1, '_suid': 266491286929, 'components': [{'_v': 1, 'name': 'source_name', '_suid': 1978769}, {'_v': 1, 'name'...",facebook_page_brand_map__003,production,2019-08-21 21:00:16.652084,production,hive://apollo_normalized.facebook_page_brand_map__3_0,production
9,facebook_page__3_0,production,2019-08-21 21:00:17.223643,2022-03-08 14:21:54.528061,"{'_v': 1, '_suid': 266491286929, 'components': [{'_v': 1, 'name': 'source_name', '_suid': 1978769}, {'_v': 1, 'name'...",facebook_page__003,production,2019-08-21 21:00:17.223643,production,hive://apollo_normalized.facebook_page__3_0,production
10,facebook_page__ts__1_0,production,2019-08-21 21:00:17.793507,2022-03-08 14:22:02.971669,"{'_v': 1, '_suid': 115392967149, 'components': [{'_v': 1, 'name': 'execution_period', '_suid': 6968176}], 'path_temp...",facebook_page__ts__001,production,2019-08-21 21:00:17.793507,production,hive://apollo_normalized.facebook_page__ts__1_0,production
12,twitter__ts_interpolated__2_0,production,2019-08-23 21:09:20.157892,2022-03-08 14:22:31.331086,"{'_v': 1, '_suid': 115392967149, 'components': [{'_v': 1, 'name': 'execution_period', '_suid': 6968176}], 'path_temp...",twitter__ts_interpolated__002,production,2019-08-26 19:42:24.793866,production,hive://apollo_calculated.twitter__ts_interpolated__2_0,production
19,instagram__ts__1_0,production,2019-08-27 19:52:08.215412,2022-03-08 14:21:31.052895,"{'_v': 1, '_suid': 115392967149, 'components': [{'_v': 1, 'name': 'execution_period', '_suid': 6968176}], 'path_temp...",instagram__ts__001,production,2019-08-30 19:01:16.462376,production,hive://apollo_normalized.instagram__ts__1_0,production
20,instagram_account__1_0,production,2019-08-27 18:37:05.047392,2022-03-08 14:21:25.336338,"{'_v': 1, '_suid': 115392967150, 'components': [{'_v': 1, 'name': 'execution_date', '_suid': 6968177}], 'path_templa...",instagram_account__001,production,2019-08-30 19:01:19.560090,production,hive://apollo_normalized.instagram_account__1_0,production
21,twitter_account__1_0,production,2019-08-22 20:01:39.149643,2022-03-08 14:22:25.749981,"{'_v': 1, '_suid': 115392967150, 'components': [{'_v': 1, 'name': 'execution_date', '_suid': 6968177}], 'path_templa...",twitter_account__001,production,2019-08-22 21:55:10.517000,production,hive://apollo_normalized.twitter_account__1_0,production
22,instagram_brand_map__1_0,production,2019-08-27 19:42:55.933257,2022-03-08 14:21:28.228613,"{'_v': 1, '_suid': 115392967150, 'components': [{'_v': 1, 'name': 'execution_date', '_suid': 6968177}], 'path_templa...",instagram_brand_map__001,production,2019-08-30 19:01:20.119343,production,hive://apollo_normalized.instagram_brand_map__1_0,production


# Reviews

In [3]:
# All of our brands live in a service called the Brand service. We can access these brands through Kibana, the brand platform tool and s3.

In [4]:
REV_COLS = [
    'source_name',
    'normalized_url',
    'review_source_id',
    'reviewer_source_id',
    'brand_name',
    'product_name',
    'product_source_id',
    'review_date',
    'review_rating',
    'review_content'
]

reviews = (
    apollo
    .dataset('online_reviews__deduped_review__1_0')
    .latest_segment_df()
    .select(REV_COLS)
    .persist()
)

In [5]:
reviews.printSchema()

root
 |-- source_name: string (nullable = true)
 |-- normalized_url: string (nullable = true)
 |-- review_source_id: string (nullable = true)
 |-- reviewer_source_id: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_source_id: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_rating: float (nullable = true)
 |-- review_content: string (nullable = true)



In [6]:
#spark_to_pandas(reviews)

A simple way to identify brands in the system is by using their url. We use the canonical_url as the main name for the brand 
in Helio but there might be other urls associated with a brand. The main id in the system currently is this **canonical_url** but we 
are migrating to use the brand_id.

In [7]:
filtered_revs = reviews.where(c.col('normalized_url')=='jot.co')


TypeError: 'Column' object is not callable

In [ ]:
filtered_revs.show()